In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
df = pd.read_parquet("../data/sc.parquet")
print(df.shape)
print(df.info())

In [ ]:
# Conversoes de tipo
df['vacina_dataAplicacao'] = pd.to_datetime(df['vacina_dataAplicacao'], errors='coerce')
df['paciente_dataNascimento'] = pd.to_datetime(df['paciente_dataNascimento'], errors='coerce')

In [ ]:
# Criar coluna de idade corrigindo erros
df['idade_calculada'] = (df['vacina_dataAplicacao'] - df['paciente_dataNascimento']).dt.days // 365
df['idade_calculada'] = df['idade_calculada'].fillna(df['paciente_idade'])

In [ ]:
# Corrigir nomes de colunas com erro de digitacao
df.rename(columns={'estalecimento_noFantasia': 'estabelecimento_noFantasia'}, inplace=True)

In [ ]:
# Corrigir strings inconsistentes
df['paciente_enumSexoBiologico'] = df['paciente_enumSexoBiologico'].str.upper().replace({
    'MASCULINO': 'M', 'FEMININO': 'F', 'M': 'M', 'F': 'F'
})

In [ ]:
# Remover duplicados e valores ausentes
df = df.drop_duplicates(subset=['document_id'])
df = df.dropna(subset=['vacina_dataAplicacao', 'vacina_nome'])

In [ ]:
# Filtrar apenas SC (garantia, pois temos mais alem de SC)
df = df[df['paciente_endereco_uf'] == 'SC']

In [ ]:
# Criar colunas derivadas
df['ano_aplicacao'] = df['vacina_dataAplicacao'].dt.year
df['mes_aplicacao'] = df['vacina_dataAplicacao'].dt.month
df['faixa_etaria'] = pd.cut(df['idade_calculada'],
                            bins=[0, 12, 18, 30, 45, 60, 80, 120],
                            labels=['0-12', '13-18', '19-30', '31-45', '46-60', '61-80', '80+'])

In [ ]:
df.to_parquet('../data/sc_pre_processado.parquet', engine='fastparquet')